In [30]:
import os
import librosa
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Set the path to the audio files
data_path = "TurkishAcademy_VC_2022"
test_path = "TurkishAcademy_VC_2022/Test"

# List all the class labels (folder names)
class_labels = ["0_AZIZSANCAR", "1_BIYKEMBOZKURT", "2_CAHITARF", "3_CANANDAGDEVIREN", "4_KORAYKAVUKCUOGLU"]

# Initialize lists to store audio features and corresponding labels
features = []
labels = []

# Extract features from training audio files
for label, folder in enumerate(class_labels):
    folder_path = os.path.join(data_path, folder)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            # Skip directories
            if os.path.isdir(file_path):
                continue
            # Load audio file
            audio, sr = librosa.load(file_path, duration=5.0)
            # Extract features (e.g., using MFCC)
            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
            # Flatten the feature matrix
            mfcc_flat = mfcc.flatten()
            # Append the features and label to the lists
            features.append(mfcc_flat)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

# Convert the lists to numpy arrays
X_train = np.array(features)
y_train = np.array(labels)

# Initialize lists to store test audio features and corresponding labels
test_features = []
test_labels = []

# Extract features from test audio files
for file_name in os.listdir(test_path):
    file_path = os.path.join(test_path, file_name)
    try:
        # Skip directories
        if os.path.isdir(file_path):
            continue
        # Load audio file
        audio, sr = librosa.load(file_path, duration=5.0)
        # Extract features (e.g., using MFCC)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        # Flatten the feature matrix
        mfcc_flat = mfcc.flatten()
        # Append the features and label to the lists
        test_features.append(mfcc_flat)
        test_labels.append(file_name)  # Store the file name as the label for evaluation purposes
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Convert the test lists to numpy arrays
X_test = np.array(test_features)
y_test = np.array(test_labels)

# Ensure that we have enough data for training and testing
if len(class_labels) <= 1 or len(X_train) <= 1:
    raise ValueError("Insufficient data for training and testing.")

# Train a support vector machine (SVM) classifier
classifier = SVC()
classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.0


/home/cedan/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


In [7]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Function to extract features from audio files
def extract_features(file_path):
    try:
        audio, _ = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=40).T, axis=0)
    except Exception as e:
        print("Error encountered while parsing audio file:", file_path)
        return None, None
    return mfccs, os.path.basename(os.path.dirname(file_path))

# Function to traverse through the dataset folder and extract features
def load_data(dataset_path):
    features = []
    labels = []
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                if file_path.endswith(".wav"):
                    feature, label = extract_features(file_path)
                    if feature is not None and label is not None:
                        features.append(feature)
                        labels.append(label)
    return np.array(features), np.array(labels)

# Load the training dataset
dataset_path = "TurkishAcademy_VC_2022"
X, y = load_data(dataset_path)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a support vector machine (SVM) classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Load the test dataset
test_dataset_path = "TurkishAcademy_VC_2022/Test"
test_files = []
for folder in os.listdir(test_dataset_path):
    folder_path = os.path.join(test_dataset_path, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if file_path.endswith(".wav"):
                test_files.append(file_path)
# Extract features from the test files
test_features = []
test_filenames = []
for file_path in test_files:
    feature, filename = extract_features(file_path)
    if feature is not None and filename is not None:
        test_features.append(feature)
        test_filenames.append(filename)
    else:
        print("Skipping file:", file_path)

if not test_features:
    print("No valid test features found. Please check the Test folder and the audio files.")
else:
    # Convert the test features to a 2D array
    test_features = np.array(test_features)

    # Reshape the test features to a 2D array
    test_features = test_features.reshape(len(test_features), -1)

    # Perform prediction on the test features
    test_predictions = svm_classifier.predict(test_features)

    # Decode the predicted labels
    predicted_labels = label_encoder.inverse_transform(test_predictions)

    # Generate the output CSV file
    output_data = {"FileName": test_filenames, "Class": predicted_labels}
    output_df = pd.DataFrame(output_data)
    output_df.to_csv("output.csv", index=False)


Accuracy: 0.7630922693266833
No valid test features found. Please check the Test folder and the audio files.


In [7]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Set the path to the audio folders
audio_folder = ''
test_folder = 'Test'

# Get the list of audio files and their corresponding labels
audio_files = []
labels = []
for folder_name in ['0_AZIZSANCAR', '1_BIYKEMBOZKURT', '2_CAHITARF', '3_CANANDAGDEVIREN', '4_KORAYKAVUKCUOGLU']:
    folder_path = os.path.join(audio_folder, folder_name)
    for file_name in os.listdir(folder_path):
        audio_files.append(os.path.join(folder_path, file_name))
        labels.append(int(folder_name.split('_')[0]))

# Extract features from the audio files
features = []
for audio_file in audio_files:
    audio, _ = librosa.load(audio_file, duration=5)  # Load audio file
    mfcc = librosa.feature.mfcc(y=audio)  # Extract MFCC features
    features.append(np.mean(mfcc, axis=1))  # Take the mean of MFCC coefficients

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Train a Support Vector Machine (SVM) classifier
svm = SVC()
svm.fit(X_train, y_train)

# Predict the labels for the test data
test_files = []
test_features = []
for file_name in os.listdir(test_folder):
    test_files.append(file_name)
    audio, _ = librosa.load(os.path.join(test_folder, file_name), duration=5)  # Load audio file
    mfcc = librosa.feature.mfcc(y=audio)  # Extract MFCC features
    test_features.append(np.mean(mfcc, axis=1))  # Take the mean of MFCC coefficients

predictions = svm.predict(test_features)
predicted_labels = label_encoder.inverse_transform(predictions)

# Write the output to a CSV file
output_data = pd.DataFrame({'FileName': test_files, 'Class': predicted_labels})
output_data.to_csv('output.csv', index=False)
